In [1]:
# Imports:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import sklearn
from sklearn.metrics import mean_squared_error
import plotly.express as px
from scipy import stats
import torch
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader

In [2]:
#set seed for reproducibility
random_state = 42
np.random.seed(random_state)

--------

### Horizon Analyses:

In [3]:
# import horizon files 
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model/data/hor1_test_data_dict.pickle', 'rb') as f:
    hor_1_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model/data/hor2_test_data_dict.pickle', 'rb') as f:
    hor_2_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model/data/hor3_test_data_dict.pickle', 'rb') as f:
    hor_3_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model/data/hor4_test_data_dict.pickle', 'rb') as f:
    hor_4_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model/data/hor8_test_data_dict.pickle', 'rb') as f:
    hor_8_test_dict = pickle.load(f)

In [4]:
hor_1_test_dict['All items']

,Inflation t-12,Inflation t-11,Inflation t-10,Inflation t-9,Inflation t-8,Inflation t-7,Inflation t-6,Inflation t-5,Inflation t-4,Inflation t-3,Inflation t-2,Inflation t-1,Inflation t,Inflation t+1
54293,0.174044,0.366634,0.188816,0.044570,0.203120,0.369596,0.584957,0.563764,-0.052378,-0.798284,-0.424370,0.088825,0.123328,0.354022
54294,0.354022,0.174044,0.366634,0.188816,0.044570,0.203120,0.369596,0.584957,0.563764,-0.052378,-0.798284,-0.424370,0.088825,0.618238
54295,0.618238,0.354022,0.174044,0.366634,0.188816,0.044570,0.203120,0.369596,0.584957,0.563764,-0.052378,-0.798284,-0.424370,0.767086
54296,0.767086,0.618238,0.354022,0.174044,0.366634,0.188816,0.044570,0.203120,0.369596,0.584957,0.563764,-0.052378,-0.798284,0.642159
54297,0.642159,0.767086,0.618238,0.354022,0.174044,0.366634,0.188816,0.044570,0.203120,0.369596,0.584957,0.563764,-0.052378,0.900787
54298,0.900787,0.642159,0.767086,0.618238,0.354022,0.174044,0.366634,0.188816,0.044570,0.203120,0.369596,0.584957,0.563764,0.472715
54299,0.472715,0.900787,0.642159,0.767086,0.618238,0.354022,0.174044,0.366634,0.188816,0.044570,0.203120,0.369596,0.584957,0.273989
54300,0.273989,0.472715,0.900787,0.642159,0.767086,0.618238,0.354022,0.174044,0.366634,0.188816,0.044570,0.203120,0.369596,0.411588
54301,0.411588,0.273989,0.472715,0.900787,0.642159,0.767086,0.618238,0.354022,0.174044,0.366634,0.188816,0.044570,0.203120,0.938899
54302,0.938899,0.411588,0.273989,0.472715,0.900787,0.642159,0.767086,0.618238,0.354022,0.174044,0.366634,0.188816,0.044570,0.776096


In [5]:
import sys        
sys.path.append('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model')  

from GRU_model import *
from config import * 

Lr = 0.001


In [6]:
def load_checkpoint(checkpoint_path, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_path)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min

In [7]:
Device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_forecast_steps = 8

category_horizon_dict = {}

for category_name in list(hor_1_test_dict.keys()):
    # load model
    cat_model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
    cat_optimizer = torch.optim.AdamW(cat_model.parameters(), lr=Lr)
    cat_model.to(Device)

    try:
        category_model, optimizer, checkpoint, valid_loss_min = load_checkpoint('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/basic_model/checkpoints/best_checkpoints/'+category_name+'.pt', cat_model, cat_optimizer)

    except:
        print(f'{category_name} not found')

    horizon_dict = {}

    # Generate forecasts step by step
    for step in range(num_forecast_steps):
        with torch.no_grad():
            # create dataloader
            x_test = hor_1_test_dict[category_name].iloc[:,:-1].to_numpy()
            y_test = hor_1_test_dict[category_name].iloc[:,-1:].to_numpy()

            x_test = torch.from_numpy(x_test).to(torch.float32)
            y_test = torch.from_numpy(y_test).to(torch.float32)

            test_dataset = TensorDataset(x_test, y_test)

            test_dataloader =  DataLoader(test_dataset, batch_size=33, shuffle=False) #### NOTE: I had to fix the batch size to 33 because of size mismatch issues (input was larger than batch size and it caused issues - so it is 33 rather than 32 like the rest (canada and norway datasets))

            for inputs, labels in test_dataloader:
                # since each batch contains 32 samples, and the largest df is of size 32, then we dont need to concat the outputs per batch per category.
                inputs = inputs.view(inputs.shape[0], SequenceLength, Features)
                #print(f'inputs shape: {inputs.shape}')
                #print(f'labels shape: {labels.shape}')
                inputs, labels = inputs.to(Device), labels.to(Device)
                print(f'inputs shape: {inputs.shape}')
                out = category_model(inputs)
                out_df = pd.DataFrame(out)
                
                first = hor_1_test_dict[category_name].iloc[:,1:].reset_index(drop = True).copy()
                print(f'first shape: {first.shape}')
                second = out_df.reset_index(drop = True).rename(columns = {0: 'Inflation t+'+str(2+step)})
                print(f'second shape: {second.shape}')

                hor_1_test_dict[category_name] = pd.concat([first, second],axis=1).reset_index(drop = True) #prediction
                print(f'hor_1_test_dict shape: {hor_1_test_dict[category_name].shape}')
                horizon_dict[step+2] = hor_1_test_dict[category_name]

    category_horizon_dict[category_name] = horizon_dict
    


inputs shape: torch.Size([33, 13, 1])
first shape: (33, 13)
second shape: (33, 1)
hor_1_test_dict shape: (33, 14)
inputs shape: torch.Size([33, 13, 1])
first shape: (33, 13)
second shape: (33, 1)
hor_1_test_dict shape: (33, 14)
inputs shape: torch.Size([33, 13, 1])
first shape: (33, 13)
second shape: (33, 1)
hor_1_test_dict shape: (33, 14)
inputs shape: torch.Size([33, 13, 1])
first shape: (33, 13)
second shape: (33, 1)
hor_1_test_dict shape: (33, 14)
inputs shape: torch.Size([33, 13, 1])
first shape: (33, 13)
second shape: (33, 1)
hor_1_test_dict shape: (33, 14)
inputs shape: torch.Size([33, 13, 1])
first shape: (33, 13)
second shape: (33, 1)
hor_1_test_dict shape: (33, 14)
inputs shape: torch.Size([33, 13, 1])
first shape: (33, 13)
second shape: (33, 1)
hor_1_test_dict shape: (33, 14)
inputs shape: torch.Size([33, 13, 1])
first shape: (33, 13)
second shape: (33, 1)
hor_1_test_dict shape: (33, 14)
inputs shape: torch.Size([33, 13, 1])
first shape: (33, 13)
second shape: (33, 1)
hor_1_

In [8]:
def avg_rmse_hor(prediction_dict, actual_dict, horizon):
    mse_lst = []
    for key in list(prediction_dict.keys()):
        if prediction_dict[key] == {}:
            continue
        predictions = prediction_dict[key][horizon].iloc[:,-1:].reset_index(drop = True)
        actuals = actual_dict[key].iloc[:,-1:].reset_index(drop = True)

        y_pred = predictions.values
        y_actual = actuals.values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        #print(f'y pred: {predicted_values_non_null}')
        #print(f'y actual: {observed_values_non_null}')
        mse = mean_squared_error(predicted_values_non_null, observed_values_non_null)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'MSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

In [9]:
avg_rmse_hor(category_horizon_dict, hor_2_test_dict, 2)

RMSE:  1.3558333519163606
MSE std:  1.1732734073611837
interval: [0.18255994455517688, 2.5291067592775445]


(1.3558333519163606, 1.1732734073611837)

In [11]:
avg_rmse_hor(category_horizon_dict, hor_3_test_dict, 3)

RMSE:  1.3613139658011646
MSE std:  1.1620962344574388
interval: [0.19921773134372578, 2.5234102002586036]


(1.3613139658011646, 1.1620962344574388)

In [12]:
avg_rmse_hor(category_horizon_dict, hor_4_test_dict, 4)

RMSE:  1.360060081384894
MSE std:  1.16971056813452
interval: [0.19034951325037408, 2.529770649519414]


(1.360060081384894, 1.16971056813452)

In [13]:
avg_rmse_hor(category_horizon_dict, hor_8_test_dict, 8)

RMSE:  1.3819171291257548
MSE std:  1.214291785532034
interval: [0.16762534359372072, 2.596208914657789]


(1.3819171291257548, 1.214291785532034)

In [14]:
#category_horizon_dict - prediction dict per category per horizon
# us_hor_x_test_dict - actual dict per category for horizon x

In [15]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_2_period_dataset_dict.pickle', 'rb') as f:
    hor_2_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_3_period_dataset_dict.pickle', 'rb') as f:
    hor_3_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_4_period_dataset_dict.pickle', 'rb') as f:
    hor_4_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_8_period_dataset_dict.pickle', 'rb') as f:
    hor_8_raw_dataset_dict = pickle.load(f)

In [16]:
def create_test_dataframe(raw_dataset_dict: dict, horizon:int):
    test_dict = {}
    for key in raw_dataset_dict.keys():
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Inflation t+'+str(horizon)]]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+'+str(horizon): 'Actual Horizon '+str(horizon)}, inplace=True)
        target_df = df[df['Year'] > Year]
        target_df = target_df.reset_index(drop = True)
        test_dict[key] = target_df
    return test_dict

In [17]:
test_dict_hor_2 = create_test_dataframe(hor_2_raw_dataset_dict, 2)
test_dict_hor_3 = create_test_dataframe(hor_3_raw_dataset_dict, 3)
test_dict_hor_4 = create_test_dataframe(hor_4_raw_dataset_dict, 4)
test_dict_hor_8 = create_test_dataframe(hor_8_raw_dataset_dict, 8)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_21467/3575878633.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_21467/3575878633.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+'+str(horizon): 'Actual Horizon '+str(horizon)}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_21467/3575878633.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [18]:
def get_df_with_predictions_horizon(prediction_dic: dict, actual_dic:dict, horizon: int) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        if prediction_dic[key] == {}:
            continue
        predictions = prediction_dic[key][horizon].iloc[:,-1:].reset_index(drop = True)
        predictions = predictions.rename(columns = {f'Inflation t+{horizon}': f'Prediction Horizon {horizon}'})
        actuals = actual_dic[key]

        data = pd.concat([predictions, actuals], axis=1).loc[:,['Category','Date','Year',f'Prediction Horizon {horizon}', f'Actual Horizon {horizon}']]
        all_data_dict[key] = data
        
    return all_data_dict

In [19]:
all_data_test_dict_hor_2 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_2,2)
all_data_test_dict_hor_3 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_3,3)
all_data_test_dict_hor_4 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_4,4)
all_data_test_dict_hor_8 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_8,8)

In [21]:
all_data_test_dict_hor_2['All items']

,Category,Date,Year,Prediction Horizon 2,Actual Horizon 2
0,All items,2021-01-15,2021.0,0.172381,0.618238
1,All items,2021-02-15,2021.0,0.124744,0.767086
2,All items,2021-03-15,2021.0,0.173653,0.642159
3,All items,2021-04-15,2021.0,0.410832,0.900787
4,All items,2021-05-15,2021.0,0.585946,0.472715
5,All items,2021-06-15,2021.0,0.550207,0.273989
6,All items,2021-07-15,2021.0,0.418335,0.411588
7,All items,2021-08-15,2021.0,0.317750,0.938899
8,All items,2021-09-15,2021.0,0.299540,0.776096
9,All items,2021-10-15,2021.0,0.432506,0.469350


In [22]:
def plot_results_horizon(all_data_dict, categories, horizon):
    #category_samples = random.sample(categories, 5)+['All items']
    category_samples = categories
    for category in category_samples:
        category_df = all_data_dict[category]
        y_pred = category_df[f'Prediction Horizon {horizon}'].values
        y_actual = category_df[f'Actual Horizon {horizon}'].values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        mse = mean_squared_error(predicted_values_non_null, observed_values_non_null)
        print(f'Category is: {category}')
        print(f'RMSE is: {np.sqrt(mse)}')

        fig = px.line(category_df, x="Date", y=[f'Actual Horizon {horizon}', f'Prediction Horizon {horizon}'], title=f'{category} - Actual VS Prediction: horizon {horizon}')
        fig.show()



In [23]:
def total_corr_horizon(all_data_test_dict, horizon):
    corr_dict = {}
    for key in list(all_data_test_dict.keys()):
        df = all_data_test_dict[key]
        y_pred = df[f'Prediction Horizon {horizon}'].values
        y_actual = df[f'Actual Horizon {horizon}'].values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        corr = stats.pearsonr(predicted_values_non_null,observed_values_non_null)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    print(f'Total Corr Horizon {horizon}, {total_corr}')
    return total_corr

In [24]:
total_corr_horizon(all_data_test_dict_hor_2, 2)
total_corr_horizon(all_data_test_dict_hor_3, 3)
total_corr_horizon(all_data_test_dict_hor_4, 4)
total_corr_horizon(all_data_test_dict_hor_8, 8)

Number of categories with High Correlation: 16
Total Corr Horizon 2, 12.678965089146503
Number of categories with High Correlation: 6
Total Corr Horizon 3, -6.27126118623343
Number of categories with High Correlation: 2
Total Corr Horizon 4, -5.256263076404961
Number of categories with High Correlation: 8
Total Corr Horizon 8, -8.648799210938956


-8.648799210938956

In [27]:
cats = list(all_data_test_dict_hor_2.keys())
categories = random.sample(cats, 5)+['All items']

In [28]:
#categories = list(all_data_test_dict_hor_2.keys())
plot_results_horizon(all_data_test_dict_hor_2, categories, 2)

Category is: Tires
RMSE is: 1.0258057112270915


Category is: Parking fees and tolls
RMSE is: 0.6879576818439997


Category is: Tuition, other school fees, and childcare
RMSE is: 0.13993371430486198


Category is: Sporting goods
RMSE is: 0.9309593493157509


Category is: Bread other than white
RMSE is: 1.1277529311965724


Category is: All items
RMSE is: 0.4246638603872704


In [29]:
#categories = list(all_data_test_dict_hor_3.keys())
plot_results_horizon(all_data_test_dict_hor_3, categories, 3)

Category is: Tires
RMSE is: 1.0056107639107639


Category is: Parking fees and tolls
RMSE is: 0.669684995974635


Category is: Tuition, other school fees, and childcare
RMSE is: 0.12627764956130094


Category is: Sporting goods
RMSE is: 0.9318876667470617


Category is: Bread other than white
RMSE is: 1.161072588808707


Category is: All items
RMSE is: 0.4265641961894422


In [30]:
#categories = list(all_data_test_dict_hor_4.keys())
plot_results_horizon(all_data_test_dict_hor_4, categories, 4)

Category is: Tires
RMSE is: 0.974600808548532


Category is: Parking fees and tolls
RMSE is: 0.6913902440387253


Category is: Tuition, other school fees, and childcare
RMSE is: 0.12440662564154568


Category is: Sporting goods
RMSE is: 0.9278042170586825


Category is: Bread other than white
RMSE is: 1.1605616317732648


Category is: All items
RMSE is: 0.3780598065007431


In [31]:
#categories = list(all_data_test_dict_hor_8.keys())
plot_results_horizon(all_data_test_dict_hor_8, categories, 8)

Category is: Tires
RMSE is: 0.9746568475534616


Category is: Parking fees and tolls
RMSE is: 0.6820970954438068


Category is: Tuition, other school fees, and childcare
RMSE is: 0.1302006635071587


Category is: Sporting goods
RMSE is: 0.9101580656268262


Category is: Bread other than white
RMSE is: 1.1983835192574548


Category is: All items
RMSE is: 0.41494359496769534
